In [1]:
# run 
# jupyter nbconvert --to script 2d_linear_confidenceintervals.ipynb
# to convert to .py strict and run the .py!
import scipy.stats as ss
import os
import numpy as np
import matplotlib.pyplot as plt
import argparse
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, GridSearchCV
from sklearn.svm import SVR  # for building SVR model
import pickle

import sys
sys.path.insert(0,'..')

from BMR.bmr import *
from pyearth import Earth
import pandas as pd

In [2]:
n_jobs = -1

def gen_model(X, a, b, c, d, e):
    return a*X[:, 0] + b*X[:, 1] + c*X[:, 0]**2 + d*X[:, 1]**2 + e*X[:, 0]*X[:, 1]
def gen_data(n, a, b, c, d, e, eps):
    if Xdist == 'U':
        rng = ss.uniform(loc=-4, scale=8)
    if Xdist == 'N':
        rng = ss.norm()
    X = rng.rvs(size=(n, 2))
    y = gen_model(X=X, a=a, b=b, c=c, d=d, e=e)
    if eps>0:
        y += ss.norm(loc=0, scale=eps).rvs(size=(n, ))
    y = y[:, np.newaxis]
    return X, y

def get_mars_params(x, y):
    # this below is not working giving some segfaults
    #param_grid = {"max_terms": [1, 2, 3, 5, 10], "max_degree": [1, 2, 3, 4, 5]}
    #mars = Earth()
    #sh = HalvingGridSearchCV(mars, param_grid, cv=3, factor=3, n_jobs=n_jobs).fit(x, y)
    #sh = GridSearchCV(mars, param_grid, cv=3, n_jobs=n_jobs).fit(x, y)
    #return sh.best_params_
    return {"max_terms": 5, "max_degree": 2}

def get_svr_params(x, y):
    param_grid = {"C": [0.1, 1, 10, 100, 300, 500, 750, 1000, 1500, 2000, 3000], "degree": [1, 2, 3, 4],
                  "epsilon": [0.01, 0.1, 1, 10]}
    svr = SVR(kernel="rbf")
    sh = HalvingGridSearchCV(svr, param_grid, cv=5, factor=3, n_jobs=1).fit(x, y)
    return sh.best_params_

def get_bmr_params(x, y, M, degree, substitution_policy="global", in_ball_model='linear'):
    n_pts = x.shape[0]
    std = np.std(x)
    param_grid = {
        #"epsilon": [0.1, 0.2, 0.3, 0.5, 0.75, 1.0, 2.0, 5.0],
        #"min_n_pts": [2, 5, 10, 25, 50, 100],
        "epsilon": [std*x for x in [0.3, 0.5]],
        "min_n_pts": [1]+[int(n_pts*x) for x in [0.01, 0.05]]
    }
    bmr = BMR(min_n_pts=10, M=M, substitution_policy=substitution_policy, 
              degree=degree, epsilon=0.5, in_ball_model=in_ball_model)
    #sh = HalvingGridSearchCV(bmr, param_grid, cv=3, factor=3, n_jobs=n_jobs).fit(x, y)
    sh = GridSearchCV(bmr, param_grid, cv=5, n_jobs=n_jobs, verbose=3).fit(x, y)
    params = sh.best_params_
    params['M'] = M
    params['substitution_policy'] = substitution_policy
    params['in_ball_model'] = in_ball_model
    print(params)
    return params

In [3]:
# generate points in which prediction is made
grid_points = np.arange(-3, 3.1, 0.2)
mesh_X, mesh_Y = np.meshgrid(grid_points, grid_points)
mesh_pts = np.array([np.ravel(mesh_X), np.ravel(mesh_Y)]).transpose()

# Some diagnostics
1. In case of trully linear model with no noise expect coefficiants in all balls to be the same close to model values

In [ ]:
# n=500
# a=1
# b=1
# c=0
# d=0
# e=0
# eps=0.00
# Xdist = 'U'
# X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
# bmr = BMR(min_n_pts=10, M=2, epsilon=0.5)
# bmr.fit(X, y)
# # investigate the bmr structure
# bmr.summary()

In [4]:
n=250
a=1
b=3
c=0.2
d=0.4
e=1
eps=0.01
Xdist='U'

In [ ]:
# check hyperparameters for BMR
# global substitution
for loops in range(10):
    X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
    params = get_bmr_params(X, y, M=10, degree=1, substitution_policy='global')
    print(params)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


In [ ]:
# nearest substitution
for loops in range(10):
    X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
    params = get_bmr_params(X, y, M=10, degree=1, substitution_policy="nearest")
    print(params)

In [5]:
def run_experiment(n, a, b, c, d, e, eps, mcloops=100, substitution_policy='global'):
    alpha = 0.05
    filename = f'CI_n={n}_a={a}_b={b}_c={c}_d{d}_e{e}_eps={eps}_{substitution_policy}_X{Xdist}.csv'
    filename_params = f'CI_n={n}_a={a}_b={b}_c={c}_d{d}_e{e}_eps={eps}_{substitution_policy}_X{Xdist}_hyperparams.pickle'
    
    X_pred = mesh_pts
    y_true = gen_model(X_pred, a, b, c, d, e)

    #generate one sample to set method parameters
    X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
    bmr_params = get_bmr_params(X, y, M=20, degree=1, substitution_policy=substitution_policy)
    mars_params = get_mars_params(X, y[:, 0])
    svr_params = get_svr_params(X, y[:, 0])
    
    # save params to file
    pickle.dump([bmr_params, mars_params, svr_params], open(filename_params, 'wb'))
    #pickle.dump([bmr_params, svr_params], open(filename_params, 'wb'))
    
    # init methods
    methods_labels = ['LR', 'BMR', 'MARS', 'SVR']
    #methods_labels = ['LR', 'BMR', 'SVR']
    
    results = {}
    betas = {}
    intercepts = {}
    for method_label in methods_labels:
        results[method_label] = []
        betas[method_label] = []
        intercepts[method_label] = []

    for loop in range(mcloops):
        #if loop % 10 == 0:
        print(f'Running loop {loop}/{mcloops} for {filename}')
        
        # run all methods on new data set
        X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
        methods = [LinearRegression(), BMR(**bmr_params), Earth(**mars_params), SVR(**svr_params)]
        #methods = [LinearRegression(), BMR(**bmr_params), SVR(**svr_params)]
        for method_label, method in zip(methods_labels, methods):
            if method_label == 'SVR':
                method.fit(X, y[:, 0])
            else:
                method.fit(X, y)
            pred = method.predict(X_pred)
            if len(pred.shape) > 1:
                pred = pred[:, 0]
            results[method_label].append(pred)
            # save coefficients
            if method_label == 'BMR':
                beta, intercept = method.coefficients(X_pred)
                betas['BMR'].append(beta)
                intercepts['BMR'].append(intercept)
            if method_label == 'LR':
                betas['LR'].append(method.coef_)
                intercepts['LR'].append(method.intercept_)
    coeff = {}
    coeff['BMR_beta1_low'] = np.quantile(np.array(betas['BMR']), q=alpha/2, axis=0)[:, 0] 
    coeff['BMR_beta1_up'] = np.quantile(np.array(betas['BMR']), q=1-alpha/2, axis=0)[:, 0]
    coeff['BMR_beta1_len'] = np.array(coeff['BMR_beta1_up']) - np.array(coeff['BMR_beta1_low'])
    coeff['BMR_beta2_low'] = np.quantile(np.array(betas['BMR']), q=alpha/2, axis=0)[:, 1] 
    coeff['BMR_beta2_up'] = np.quantile(np.array(betas['BMR']), q=1-alpha/2, axis=0)[:, 1]
    coeff['BMR_beta2_len'] = np.array(coeff['BMR_beta2_up']) - np.array(coeff['BMR_beta2_low'])
    coeff['BMR_I_low'] = np.quantile(np.array(intercepts['BMR']), q=alpha/2, axis=0) 
    coeff['BMR_I_up'] = np.quantile(np.array(intercepts['BMR']), 1-alpha/2, axis=0) 
    coeff['BMR_I_len'] = np.array(coeff['BMR_I_up']) - np.array(coeff['BMR_I_low'])
    coeff['LR_beta1_low'] = np.quantile(np.array(betas['LR']), q=alpha/2, axis=0)[:, 0].tolist()*X_pred.shape[0]
    coeff['LR_beta1_up'] = np.quantile(np.array(betas['LR']), q=1-alpha/2, axis=0)[:, 0].tolist()*X_pred.shape[0]
    coeff['LR_beta1_len'] = np.array(coeff['LR_beta1_up']) - np.array(coeff['LR_beta1_low'])
    coeff['LR_beta2_low'] = np.quantile(np.array(betas['LR']), q=alpha/2, axis=0)[:, 1].tolist()*X_pred.shape[0]
    coeff['LR_beta2_up'] = np.quantile(np.array(betas['LR']), q=1-alpha/2, axis=0)[:, 1].tolist()*X_pred.shape[0]
    coeff['LR_beta2_len'] = np.array(coeff['LR_beta2_up']) - np.array(coeff['LR_beta2_low'])
    coeff['LR_I_low'] = np.quantile(np.array(intercepts['LR']), q=alpha/2, axis=0).tolist()*X_pred.shape[0]
    coeff['LR_I_up'] = np.quantile(np.array(intercepts['LR']), q=1-alpha/2, axis=0).tolist()*X_pred.shape[0]
    coeff['LR_I_len'] = np.array(coeff['LR_I_up']) - np.array(coeff['LR_I_low'])
       
    # collect the results and prepare the csv
    df0 = pd.DataFrame([mesh_pts[:, 0], mesh_pts[:, 1]]).transpose()
    df0.columns = ['x', 'y']
    dfs = [df0]
    for method_label in methods_labels:
        dat = np.array(results[method_label]).transpose()
        ci_low = np.quantile(dat, q=alpha/2, axis=1)
        ci_up = np.quantile(dat, q=1-alpha/2, axis=1)
        mse = np.mean((dat - y_true.reshape(-1,1))**2, axis=1)
        df = pd.DataFrame([ci_low, ci_up, ci_up-ci_low, mse]).transpose()
        df.columns = [f'{method_label}_CI_low', f'{method_label}_CI_up', f'{method_label}_CI_len', f'{method_label}_MSE']
        dfs.append(df)
    # add DataFrame containing coefficients
    dfs.append(pd.DataFrame(coeff))
    df = pd.concat(dfs, axis=1)
    df.to_csv(filename, index=False)
    return df, betas, coeff

In [ ]:
df, betas, coeff = run_experiment(n=250, a=1, b=2, c=0, d=0, e=0, eps=0.01, mcloops=2, substitution_policy='global')

In [7]:
df, betas, coeff = run_experiment(n=500, a=1, b=2, c=0.2, d=0.6, e=1, eps=0.01, mcloops=1, substitution_policy='nearest')

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/home/rafal/.pyenv/versions/3.7.0/envs/venv-mars-3.7.0/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/rafal/.pyenv/versions/3.7.0/envs/venv-mars-3.7.0/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "../BMR/bmr.py", line 156, in fit
    self.__fit_nearest(xscaled, y)
  File "../BMR/bmr.py", line 123, in __fit_nearest
    raise ValueError(f'All balls in BM contain less than {self.min_n_pts} points. '
ValueError: All balls

{'epsilon': 0.6911943282319831, 'min_n_pts': 1, 'M': 20, 'substitution_policy': 'nearest', 'in_ball_model': 'linear'}
Running loop 0/1 for CI_n=500_a=1_b=2_c=0.2_d0.6_e1_eps=0.01_nearest_XU.csv


/home/rafal/.pyenv/versions/3.7.0/envs/venv-mars-3.7.0/lib/python3.7/site-packages/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
/home/rafal/.pyenv/versions/3.7.0/envs/venv-mars-3.7.0/lib/python3.7/site-packages/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


[CV 5/5] END epsilon=0.6911943282319831, min_n_pts=5;, score=0.126 total time=   4.2s
[CV 1/5] END epsilon=0.6911943282319831, min_n_pts=5;, score=0.124 total time=   4.3s
[CV 2/5] END epsilon=0.6911943282319831, min_n_pts=5;, score=0.134 total time=   4.2s
[CV 4/5] END epsilon=0.6911943282319831, min_n_pts=25;, score=nan total time=   0.1s
[CV 3/5] END epsilon=1.1519905470533052, min_n_pts=1;, score=0.227 total time=   2.0s
[CV 5/5] END epsilon=1.1519905470533052, min_n_pts=1;, score=0.379 total time=   2.1s
[CV 2/5] END epsilon=0.6911943282319831, min_n_pts=25;, score=nan total time=   0.1s
[CV 1/5] END epsilon=1.1519905470533052, min_n_pts=1;, score=0.260 total time=   2.1s
[CV 1/5] END epsilon=1.1519905470533052, min_n_pts=5;, score=0.186 total time=   2.2s
[CV 1/5] END epsilon=0.6911943282319831, min_n_pts=25;, score=nan total time=   0.1s
[CV 5/5] END epsilon=0.6911943282319831, min_n_pts=25;, score=nan total time=   0.1s
[CV 2/5] END epsilon=1.1519905470533052, min_n_pts=1;, sco

In [ ]:
# # df, betas, coeff = run_experiment(n=500, a=0, b=0, c=1, d=1, e=0, eps=0.0001, mcloops=100)
# # {'epsilon': 5.0, 'min_n_pts': 2, 'M': 20}
# # NORMAL
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [ ]:
# # df, betas, coeff = run_experiment(n=500, a=0, b=0, c=1, d=1, e=0, eps=0.0001, mcloops=100)
# # {'epsilon': 5.0, 'min_n_pts': 100, 'M': 20}
# # UNIFORM
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [ ]:
# # df, betas, coeff = run_experiment(n=500, a=0, b=0, c=1, d=1, e=0, eps=0.0001, mcloops=100)
# # {'epsilon': 0.24970063583258964, 'min_n_pts': 150, 'M': 20}
# # UNIFORM
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [ ]:
# # bmr_params = {'epsilon': 1, 'min_n_pts': 5, 'M': 10}
# # UNIFORM
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [ ]:
# pd.DataFrame.from_dict(coeff)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--n", type=int, required=True, help="sample size")
parser.add_argument("--a", type=float, required=True, help="param a")
parser.add_argument("--b", type=float, required=True, help="param b")
parser.add_argument("--c", type=float, required=True, help="param c")
parser.add_argument("--d", type=float, required=True, help="param d")
parser.add_argument("--e", type=float, required=True, help="param e")
parser.add_argument("--eps", type=float, required=True, help="noise")
parser.add_argument("--M", type=int, required=True, help="number of MC loops")
parser.add_argument("--X", type=str, required=True, help="X distribution")
args = parser.parse_args()

In [ ]:
Xdist = args.X
if Xdist not in ['U', 'N']:
    raise ValueError(f'--X must be U or N. Found {Xdist}')

run_experiment(n=args.n, a=args.a, b=args.b, c=args.c, d=args.d, e=args.e, eps=args.eps, mcloops=args.M)